In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [10]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band8
  atmpro = mls
  band = 8
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0, -1, 0]
  ng_refs = [3, 2, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 1900
  vmin = 1380
  w_diffuse = [(1.66, 1.66, 1.66), (1.66, 1.66), (1.66, 1.66, 1.66)]
  wgt = [(0.55, 0.55, 0.85), (0.85, 0.85), (0, 0.3, 0.55)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.134107                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.162498
0.000750      2.0 -0.133502                  2.0 -0.161758
0.001052      3.0 -0.130775                  3.0 -0.159758
0.001476      4.0 -0.127989                  4.0 -0.157632
0.002070      5.0 -0.125396                  5.0 -0.155543
0.002904      6.0 -0.123131                  6.0 -0.153548
0.004074      7.0 -0.121313                  7.0 -0.151679
0.005714      8.0 -0.120101                  8.0 -0.150072
0.008015      9.0 -0.119749                  9.0 -0.148785
0.011243     10.0 -0.120530                 10.0 -0.148074
0.015771     11.0 -0.122859                 11.0 -0.148097
0.022122     12.0 -0.127290                 12.0 -0.149326
0.031031     13.0 -0.134674                 13.0 -0.152218
0.043528     14.0 -0.146204                 14.0 -0.157405
0.061057     15.0 -0.154468                 15.0 -0.159235
0.085645     16.0 -0.141828                 16.0 -0.144398
0.120136     17.0 -0.107541                 17.0 -0.110849
0.168516     18.0 -0.065559                 18.0 -0.067670
0.236378     19.0 -0.016042                 19.0 -0.015404
0.331549     20.0  0.043121                 20.0  0.046859
0.465100     21.0  0.116778                 21.0  0.122272
0.652400     22.0  0.220287                 22.0  0.222158
0.915100     23.0  0.326819                 23.0  0.318884
1.283650     24.0  0.292899                 24.0  0.282336
1.800600     25.0  0.169545                 25.0  0.163651
2.525700     26.0  0.093714                 26.0  0.095586
3.542800     27.0  0.056653                 27.0  0.061831
4.969550     28.0  0.038365                 28.0  0.040805
6.970850     29.0  0.029634                 29.0  0.029649
9.778100     30.0  0.024466                 30.0  0.024588
13.715850    31.0  0.019408                 31.0  0.021083
19.239350    32.0  0.014145                 32.0  0.016288
26.987250    33.0  0.010873                 33.0  0.012691
37.855300    34.0  0.009286                 34.0  0.011125
53.100050    35.0  0.006934                 35.0  0.008238
73.887500    36.0  0.003756                 36.0  0.003462
97.662500    37.0  0.001862                 37.0  0.000744
121.437500   38.0  0.001142                 38.0  0.000358
145.212500   39.0 -0.000864                 39.0 -0.001174
168.987500   40.0 -0.003060                 40.0 -0.002993
192.762500   41.0 -0.000992                 41.0 -0.000020
216.537500   42.0  0.014138                 42.0  0.019970
240.312500   43.0  0.039087                 43.0  0.050346
264.087500   44.0  0.067254                 44.0  0.058249
287.862500   45.0  0.092458                 45.0  0.062925
311.637500   46.0  0.099109                 46.0  0.084594
335.412500   47.0  0.105350                 47.0  0.104689
359.187500   48.0  0.114194                 48.0  0.115929
382.962500   49.0  0.122338                 49.0  0.120288
406.737500   50.0  0.128138                 50.0  0.123793
430.512500   51.0  0.130195                 51.0  0.130116
454.287500   52.0  0.126481                 52.0  0.134135
478.062500   53.0  0.120707                 53.0  0.136015
501.837500   54.0  0.114206                 54.0  0.132230
525.612500   55.0  0.107169                 55.0  0.123617
549.387500   56.0  0.100231                 56.0  0.111034
573.162500   57.0  0.094581                 57.0  0.098082
596.937500   58.0  0.091548                 58.0  0.088574
620.712500   59.0  0.083912                 59.0  0.076223
644.487500   60.0  0.073717                 60.0  0.065253
668.262500   61.0  0.063024                 61.0  0.055995
692.037500   62.0  0.052869                 62.0  0.047866
715.812500   63.0  0.044022                 63.0  0.040207
739.587500   64.0  0.037069                 64.0  0.035409
76

# Fluxes by Layer

CRD                          CLIRAD                \
                         flug       fldg     fnetg       flug          fldg   
pressure     level                                                            
0.000000e+00 1      -7.292049   0.000000 -7.292049        NaN           NaN   
1.000000e-08 1            NaN        NaN       NaN  -7.160886  1.842825e-09   
6.244000e-04 2      -7.292064   0.000005 -7.292059  -7.160901  3.441324e-06   
8.759000e-04 3      -7.292070   0.000007 -7.292063  -7.160907  4.842399e-06   
1.228600e-03 4      -7.292078   0.000010 -7.292068  -7.160916  6.858441e-06   
1.723400e-03 5      -7.292090   0.000015 -7.292076  -7.160928  9.764283e-06   
2.417400e-03 6      -7.292107   0.000021 -7.292086  -7.160945  1.394981e-05   
3.390900e-03 7      -7.292130   0.000030 -7.292100  -7.160969  1.997463e-05   
4.756500e-03 8      -7.292162   0.000042 -7.292120  -7.161002  2.864095e-05   
6.672000e-03 9      -7.292207   0.000060 -7.292147  -7.161049  4.108774e-05   
9.358900e-03 10     -7.292270   0.000084 -7.292185  -7.161114  5.894095e-05   
1.312780e-02 11     -7.292358   0.000119 -7.292239  -7.161205  8.447932e-05   
1.841450e-02 12     -7.292483   0.000167 -7.292316  -7.161335  1.209119e-04   
2.583020e-02 13     -7.292660   0.000232 -7.292428  -7.161518  1.726435e-04   
3.623230e-02 14     -7.292915   0.000321 -7.292593  -7.161778  2.456788e-04   
5.082340e-02 15     -7.293286   0.000440 -7.292846  -7.162153  3.481197e-04   
7.129060e-02 16     -7.293829   0.000608 -7.293221  -7.162690  4.992906e-04   
1.000000e-01 17     -7.294596   0.000893 -7.293703  -7.163439  7.570686e-04   
1.402710e-01 18     -7.295634   0.001417 -7.294216  -7.164444  1.233568e-03   
1.967600e-01 19     -7.296999   0.002345 -7.294655  -7.165758  2.094898e-03   
2.759970e-01 20     -7.298732   0.003927 -7.294805  -7.167410  3.602201e-03   
3.871000e-01 21     -7.300786   0.006548 -7.294238  -7.169340  6.148455e-03   
5.431000e-01 22     -7.302868   0.010789 -7.292080  -7.171230  1.029888e-02   
7.617000e-01 23     -7.303848   0.017472 -7.286376  -7.171982  1.680399e-02   
1.068500e+00 24     -7.301339   0.026841 -7.274498  -7.169442  2.585436e-02   
1.498800e+00 25     -7.295243   0.035675 -7.259568  -7.163523  3.432782e-02   
2.102400e+00 26     -7.288156   0.040710 -7.247446  -7.156622  3.912906e-02   
2.949000e+00 27     -7.281011   0.042963 -7.238048  -7.149642  4.173597e-02   
4.136600e+00 28     -7.274021   0.043943 -7.230078  -7.142790  4.358360e-02   
5.802500e+00 29     -7.267257   0.044750 -7.222507  -7.136131  4.497766e-02   
8.139200e+00 30     -7.260546   0.046242 -7.214304  -7.129427  4.648151e-02   
1.141700e+01 31     -7.253652   0.048847 -7.204805  -7.122407  4.900909e-02   
1.601470e+01 32     -7.246767   0.052532 -7.194234  -7.115192  5.327724e-02   
2.246400e+01 33     -7.240745   0.057317 -7.183428  -7.108649  5.917817e-02   
3.151050e+01 34     -7.236023   0.064247 -7.171776  -7.103288  6.741886e-02   
4.420010e+01 35     -7.232438   0.074621 -7.157817  -7.099037  7.989329e-02   
6.200000e+01 36     -7.231724   0.088529 -7.143195  -7.097873  9.610156e-02   
8.577500e+01 37     -7.238364   0.105746 -7.132618  -7.104822  1.128027e-01   
1.095500e+02 38     -7.251139   0.123764 -7.127375  -7.118466  1.285429e-01   
1.333250e+02 39     -7.268015   0.143857 -7.124158  -7.136349  1.474333e-01   
1.571000e+02 40     -7.291052   0.164459 -7.126593  -7.160800  1.685787e-01   
1.808750e+02 41     -7.323783   0.188573 -7.135209  -7.195801  1.951492e-01   
2.046500e+02 42     -7.376308   0.238306 -7.138002  -7.252287  2.515775e-01   
2.284250e+02 43     -7.479923   0.381737 -7.098186  -7.361115  4.166536e-01   
2.522000e+02 44     -7.651092   0.662988 -6.988104  -7.539872  7.372150e-01   
2.759750e+02 45     -7.907381   1.108686 -6.798696  -7.807337  1.168744e+00   
2.997500e+02 46     -8.268424   1.730118 -6.538306  -8.176978  1.715622e+00   
3.235250e+02 47     -8.702170   2.442987 -6.259183  -8.615266  2.392177e+00 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')